In [1]:
# Import needed packages
import deepchem as dc 
import numpy as np
import rdkit

In [2]:
# Generate a random input and output tensor
x = np.random.random((4,5))
y = np.random.random((4,1))
print(x)
print(y)

[[0.58617281 0.8701479  0.04160662 0.63960343 0.44705335]
 [0.31865176 0.85162278 0.61855158 0.98121017 0.58804488]
 [0.81979971 0.20475982 0.31950805 0.75355156 0.54869868]
 [0.68430579 0.16361704 0.38867889 0.77749625 0.7106369 ]]
[[0.65207334]
 [0.5722105 ]
 [0.60399624]
 [0.68017512]]


In [4]:
smiles = ['C1CCCCC1', 'O1CCOCC1'] # cyclohexane and dioxane
mols = [rdkit.Chem.MolFromSmiles(smile) for smile in smiles]
feat = dc.feat.CircularFingerprint(size=1024)
arr = feat.featurize(mols)
# arr is a 2-by-1024 array containing the fingerprints for
# the two molecules

In [ ]:
arr

PDBBind dataset

In [5]:
import deepchem as dc
grid_featurizer = dc.feat.RdkitGridFeaturizer(voxel_width=2.0, feature_types=['hbond', 'salt_bridge',  'pi_stack','cation_pi', 'ecfp', 'splif'], sanitize=True, flatten=True)

In [8]:
# Download (2GB!), featurize the dataset
tasks, datasets, transformers = dc.molnet.load_pdbbind(featurizer="grid", split="random", subset="core")
train_dataset, valid_dataset, test_dataset = datasets

ok 0.165 s
TIMING: Centroid processing took 0.005 s
TIMING: Loading protein coordinates took 88.640 s
TIMING: Loading ligand coordinates took 0.251 s
TIMING: Centroid processing took 0.000 s
TIMING: Loading protein coordinates took 1544.716 s
TIMING: Loading ligand coordinates took 0.320 s
TIMING: Centroid processing took 0.002 s
TIMING: Loading protein coordinates took 85.937 s
TIMING: Loading ligand coordinates took 0.111 s
TIMING: Centroid processing took 0.001 s
TIMING: Loading protein coordinates took 18.574 s
TIMING: Loading ligand coordinates took 0.624 s
TIMING: Centroid processing took 0.000 s
TIMING: Loading protein coordinates took 10.161 s
TIMING: Loading ligand coordinates took 0.206 s
TIMING: Centroid processing took 0.000 s
TIMING: Loading protein coordinates took 11.037 s
TIMING: Loading ligand coordinates took 0.210 s
TIMING: Centroid processing took 0.000 s
TIMING: Loading protein coordinates took 18.047 s
TIMING: Loading ligand coordinates took 0.169 s
TIMING: Centro

In [9]:
# train a random forest model
from sklearn.ensemble import RandomForestRegressor
sklearn_model = RandomForestRegressor(n_estimators=100)
model = dc.models.SklearnModel(sklearn_model)
model.fit(train_dataset)

In [10]:
# train a MLP model
n_features = train_dataset.X.shape[1]
model = dc.models.MultitaskRegressor(n_tasks=len(pdbbind_tasks),n_features=n_features,layer_sizes=[2000, 1000],dropouts=0.5,learning_rate=0.0003)
model.fit(train_dataset, nb_epoch=250)

NameError: name 'pdbbind_tasks' is not defined

In [11]:
# check the model!
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)

In [12]:
print("Evaluating model")
train_scores = model.evaluate(train_dataset, [metric], transformers)
test_scores = model.evaluate(test_dataset, [metric], transformers)
print("Train scores")
print(train_scores)
print("Test scores")
print(test_scores)

Evaluating model
computed_metrics: [0.9755411721410139]
computed_metrics: [0.16437959583981504]
Train scores
{'pearson_r2_score': 0.9755411721410139}
Test scores
{'pearson_r2_score': 0.16437959583981504}
